# Ukrainian Restaurant – DimASR (Subtask 1) using BERT-Multilingual
This notebook uses `bert-base-multilingual-cased` to reduce RAM usage.
Python 3.13 compatible, CPU friendly.


In [1]:

import os, json, torch, torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from tqdm.auto import tqdm
from transformers import AutoTokenizer, AutoModel, DataCollatorWithPadding

device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
TRAIN='ukr_restaurant_train_alltasks.jsonl'
DEV='ukr_restaurant_dev_task1.jsonl'
print('CUDA:',torch.cuda.is_available())


CUDA: False


In [2]:

def read_jsonl(p):
    arr=[]
    with open(p,encoding='utf8') as f:
        for ln in f:
            ln=ln.strip()
            if ln:
                arr.append(json.loads(ln))
    print(p,'->',len(arr))
    return arr

train_json=read_jsonl(TRAIN)
dev_json=read_jsonl(DEV)


ukr_restaurant_train_alltasks.jsonl -> 1240
ukr_restaurant_dev_task1.jsonl -> 56


In [3]:

def build_train_df(data):
    rows=[]
    for ex in data:
        tid=ex['ID']; text=ex['Text']
        if 'Quadruplet' in ex:
            for q in ex['Quadruplet']:
                if 'VA' not in q: continue
                try:
                    v,a=q['VA'].split('#')
                    rows.append({
                        'ID':tid,'Text':text,'Aspect':q['Aspect'],
                        'valence':float(v),'arousal':float(a)
                    })
                except: pass
    print('TRAIN rows:',len(rows))
    return pd.DataFrame(rows)

def build_dev_df(data):
    rows=[]
    for ex in data:
        for asp in ex['Aspect']:
            rows.append({'ID':ex['ID'],'Text':ex['Text'],'Aspect':asp})
    print('DEV rows:',len(rows))
    return pd.DataFrame(rows)

train_df=build_train_df(train_json)
dev_df=build_dev_df(dev_json)

train_df.head(), dev_df.head()


TRAIN rows: 2487
DEV rows: 81


(         ID                                               Text        Aspect  \
 0   225:3_0                   Види з вікна-вище всяких похвал.  Види з вікна   
 1   225:4_1                          Інтер'єр не запам'ятався.      Інтер'єр   
 2  225:10_2  Меню здалося трохи нудним, але ми швидко визна...          Меню   
 3  225:13_3  М'ясо було просмажене трохи сильніше, ніж замо...         М'ясо   
 4  225:13_3  М'ясо було просмажене трохи сильніше, ніж замо...         М'ясо   
 
    valence  arousal  
 0     8.30     7.60  
 1     5.00     3.20  
 2     4.62     3.88  
 3     7.00     6.20  
 4     6.90     6.20  ,
          ID                                               Text   Aspect
 0  354:12_0  Блюдо хінкалі складається з 4 штук, хороших ро...  хінкалі
 1  1180:1_1  Дуже все сподобалося, музика ненав'язлива, дів...   музика
 2  1180:1_1  Дуже все сподобалося, музика ненав'язлива, дів...  дівчина
 3  1180:1_1  Дуже все сподобалося, музика ненав'язлива, дів...   порції
 4  1206

In [4]:

MODEL='bert-base-multilingual-cased'
tok=AutoTokenizer.from_pretrained(MODEL)

def combine(t,a): return f"{t} [ASP] {a}"

class DS(Dataset):
    def __init__(self,df,train=True):
        self.df=df; self.train=train
    def __len__(self): return len(self.df)
    def __getitem__(self,idx):
        r=self.df.iloc[idx]
        enc=tok(combine(r['Text'],r['Aspect']),
                truncation=True,max_length=128,return_tensors='pt')
        out={k:v.squeeze(0) for k,v in enc.items()}
        out['ID']=r['ID']; out['Aspect']=r['Aspect']
        if self.train:
            out['labels']=torch.tensor([r['valence'],r['arousal']],
                                       dtype=torch.float32)
        return out

train_ds=DS(train_df,True)
dev_ds=DS(dev_df,False)


In [5]:

coll=DataCollatorWithPadding(tok)

def collate(batch):
    IDs=[x['ID'] for x in batch]
    ASP=[x['Aspect'] for x in batch]
    for x in batch:
        x.pop('ID'); x.pop('Aspect')
    pad=coll(batch)
    pad['ID']=IDs; pad['Aspect']=ASP
    return pad

train_loader=DataLoader(train_ds,batch_size=2,shuffle=True,collate_fn=collate)
dev_loader=DataLoader(dev_ds,batch_size=4,shuffle=False,collate_fn=collate)
len(train_loader),len(dev_loader)


(1244, 21)

In [6]:

class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.base=AutoModel.from_pretrained(MODEL)
        self.reg=nn.Linear(self.base.config.hidden_size,2)
    def forward(self,i,m):
        o=self.base(input_ids=i,attention_mask=m)
        cls=o.last_hidden_state[:,0]
        return self.reg(cls)

model=Model().to(device)
opt=torch.optim.AdamW(model.parameters(),lr=2e-5)
loss_fn=nn.MSELoss()


In [7]:

EPOCHS=4
for ep in range(EPOCHS):
    model.train(); tot=0
    for b in tqdm(train_loader,desc=f'Epoch {ep+1}'):
        ids=b['input_ids'].to(device)
        mask=b['attention_mask'].to(device)
        y=b['labels'].to(device)
        pred=model(ids,mask)
        loss=loss_fn(pred,y)
        opt.zero_grad(); loss.backward(); opt.step()
        tot+=loss.item()
    print('Epoch',ep+1,'Loss',tot/len(train_loader))


Epoch 1:   0%|          | 0/1244 [00:00<?, ?it/s]

Epoch 1 Loss 2.175939302695346


Epoch 2:   0%|          | 0/1244 [00:00<?, ?it/s]

Epoch 2 Loss 1.3558854893055052


Epoch 3:   0%|          | 0/1244 [00:00<?, ?it/s]

Epoch 3 Loss 0.9494911057921726


Epoch 4:   0%|          | 0/1244 [00:00<?, ?it/s]

Epoch 4 Loss 0.7084728412787202


In [8]:

model.eval(); preds=[]
with torch.no_grad():
    for b in tqdm(dev_loader,desc='Inference'):
        ids=b['input_ids'].to(device)
        mask=b['attention_mask'].to(device)
        out=model(ids,mask).cpu().numpy()
        for i,(ID,A) in enumerate(zip(b['ID'],b['Aspect'])):
            v,a=out[i]
            preds.append((ID,A,f"{v:.2f}#{a:.2f}"))
len(preds)


Inference:   0%|          | 0/21 [00:00<?, ?it/s]

81

In [9]:

OUT='pred_ukr_restaurant.jsonl'
sub={}
for ID,A,VA in preds:
    sub.setdefault(ID,[]).append({'Aspect':A,'VA':VA})

with open(OUT,'w',encoding='utf8') as f:
    for ex in dev_json:
        f.write(json.dumps({
            'ID':ex['ID'],
            'Aspect_VA':sub.get(ex['ID'],[])
        },ensure_ascii=False)+'\n')

os.path.exists(OUT),os.path.getsize(OUT)


(True, 6285)